In [1]:
import numpy as np
import copy
customer=dict()

In [2]:
def clean(row,i):
    b=row.strip().replace("\xc2\xa0", "").replace("\xef\xbb\xbf", "").split(',')[i]
    return b
for i,row in enumerate(open('R101.csv')):
    if i >= 1:
        customer[i-1]=dict()
        customer[i-1]['X']=float(clean(row,1))
        customer[i-1]['Y']=float(clean(row,2))
        customer[i-1]['DEMAND']=float(clean(row,3))
        customer[i-1]['RTIME']=float(clean(row,4))
        customer[i-1]['DUE']=float(clean(row,5))
        customer[i-1]['STIME']=float(clean(row,6))

In [3]:
n = len(customer)
capacity = 200
M = 100000
acw = -10

In [4]:
distance_matrix = []
for i in range(n):
    distance_list = []
    for j in range(n):
        if i == j:
            distance_list.append(M)
        else:
            distance_list.append(((customer[i]['X']-customer[j]['X'])**2+(customer[i]['Y']-customer[j]['Y'])**2)**0.5)
    distance_matrix.append(distance_list)

In [5]:
cus = []
for i in range(1,n):
    cus.append(i)

In [6]:
#服務時間不可以超過due time
#從「0」開始，找最近的點連接，直到無法再連接後，新增新的「0」路線，直到cus用完
route_append = 1
route = []
leavetime = []
while len(cus) != 0:
    if (route_append == 1) & (len(cus) != 0):
        route_change = [0]
        leavetimechange = [0]
        cij = []
        route_append = 0
    else:
        pass
    #print 'cus:',cus
    #print 'route:',route
    #print 'route_change:',route_change
    #print 'leavetimechange:',leavetimechange
    cij = []
    for i in cus:
        totaldemand = 0
        for r in route_change:
            totaldemand += customer[r]['DEMAND']
        totaldemand += customer[i]['DEMAND']

        if totaldemand <= capacity:  
            if len(route_change)==1:
                waitt = leavetimechange[-1]+distance_matrix[route_change[-1]][i] - customer[i]['RTIME']
                if waitt < 0:
                    ltc = customer[i]['RTIME']+customer[i]['STIME']
                else:
                    ltc = leavetimechange[-1]+distance_matrix[route_change[-1]][i]+customer[i]['STIME']
            else:
                if leavetimechange[-1]+distance_matrix[route_change[-1]][i]+customer[i]['STIME'] > customer[i]['DUE']:                
                    #print 'over due time'
                    continue
                else:
                    waitt = leavetimechange[-1]+distance_matrix[route_change[-1]][i] - customer[i]['RTIME']
                    if (waitt < 0) & (waitt > acw):
                        ltc = customer[i]['RTIME']+customer[i]['STIME']
                    elif waitt >= 0:
                        ltc = leavetimechange[-1]+distance_matrix[route_change[-1]][i]+customer[i]['STIME']
                    else:
                        #print 'over waiting limit'
                        continue
            if ltc + distance_matrix[i][0] <= customer[0]['DUE']:
                Tij = ltc-customer[i]['STIME']-leavetimechange[-1]
                vij = customer[i]['DUE']-(leavetimechange[-1]+distance_matrix[route_change[-1]][i])
                dij = distance_matrix[route_change[-1]][i]
                cij.append([0.3*Tij+0.3*dij+0.4*vij,i,ltc])
            else:
                #print 'over depot due time'
                continue

        else:
            #print 'over capacity'
            continue
    #print 'length of cij:',len(cij)
    if len(cij) != 0:
        cij.sort()
        route_change.append(cij[0][1])
        leavetimechange.append(cij[0][2])
        cus.remove(cij[0][1])
        #print cij[0]
    else:
        route_append = 1
        route_change.append(0)
        route.append(route_change)
        leavetime.append(leavetimechange)
route_change.append(0)
route.append(route_change)
leavetime.append(leavetimechange)



In [7]:
print 'If we cannot severice over due time'
print 'Number of cars is',len(route)

print 'Routes are :'
for indexi,i in enumerate(route):
    print indexi,i

If we cannot severice over due time
Number of cars is 29
Routes are :
0 [0, 59, 95, 98, 61, 85, 26, 13, 100, 0]
1 [0, 92, 2, 73, 22, 43, 91, 58, 0]
2 [0, 42, 75, 41, 97, 60, 93, 0]
3 [0, 14, 15, 87, 56, 4, 25, 0]
4 [0, 45, 82, 53, 50, 0]
5 [0, 63, 62, 7, 46, 77, 0]
6 [0, 5, 44, 86, 74, 0]
7 [0, 72, 23, 35, 80, 0]
8 [0, 33, 29, 79, 34, 24, 0]
9 [0, 36, 64, 66, 0]
10 [0, 27, 12, 81, 3, 68, 70, 0]
11 [0, 28, 88, 55, 0]
12 [0, 39, 40, 32, 0]
13 [0, 83, 16, 84, 37, 0]
14 [0, 65, 71, 9, 20, 1, 0]
15 [0, 47, 19, 49, 17, 0]
16 [0, 31, 51, 10, 0]
17 [0, 69, 76, 0]
18 [0, 52, 99, 94, 0]
19 [0, 21, 6, 0]
20 [0, 11, 90, 0]
21 [0, 30, 0]
22 [0, 67, 48, 0]
23 [0, 38, 0]
24 [0, 18, 54, 0]
25 [0, 8, 0]
26 [0, 78, 0]
27 [0, 57, 96, 0]
28 [0, 89, 0]


In [8]:
for i,row in enumerate(open('R101.csv')):
    if i >= 1:
        customer[i-1]=dict()
        customer[i-1]['X']=float(clean(row,1))
        customer[i-1]['Y']=float(clean(row,2))
        customer[i-1]['DEMAND']=float(clean(row,3))
        customer[i-1]['RTIME']=float(clean(row,4))
        customer[i-1]['DUE']=float(clean(row,5))
        customer[i-1]['STIME']=float(clean(row,6))
n = len(customer)
capacity = 200
M = 100000
acw = -100000
distance_matrix = []
for i in range(n):
    distance_list = []
    for j in range(n):
        if i == j:
            distance_list.append(M)
        else:
            distance_list.append(((customer[i]['X']-customer[j]['X'])**2+(customer[i]['Y']-customer[j]['Y'])**2)**0.5)
    distance_matrix.append(distance_list)
cus = []
for i in range(1,n):
    cus.append(i)

In [9]:
#服務時間可以超過due time
#從「0」開始，找最近的點連接，直到無法再連接後，新增新的「0」路線，直到cus用完
route_append = 1
route = []
leavetime = []
while len(cus) != 0:
    if (route_append == 1) & (len(cus) != 0):
        route_change = [0]
        leavetimechange = [0]
        cij = []
        route_append = 0
    else:
        pass
    #print 'cus:',cus
    #print 'route:',route
    #print 'route_change:',route_change
    #print 'leavetimechange:',leavetimechange
    cij = []
    for i in cus:
        totaldemand = 0
        for r in route_change:
            totaldemand += customer[r]['DEMAND']
        totaldemand += customer[i]['DEMAND']

        if totaldemand <= capacity:  
            if leavetimechange[-1]+distance_matrix[route_change[-1]][i] > customer[i]['DUE']:                
                #print 'over due time'
                continue
            else:
                waitt = leavetimechange[-1]+distance_matrix[route_change[-1]][i] - customer[i]['RTIME']
                if (waitt < 0) & (waitt > acw):
                    ltc = customer[i]['RTIME']+customer[i]['STIME']
                elif waitt >= 0:
                    ltc = leavetimechange[-1]+distance_matrix[route_change[-1]][i]+customer[i]['STIME']
                else:
                    #print 'over waiting limit'
                    continue
            if ltc + distance_matrix[i][0] <= customer[0]['DUE']:
                Tij = ltc-customer[i]['STIME']-leavetimechange[-1]
                vij = customer[i]['DUE']-(leavetimechange[-1]+distance_matrix[route_change[-1]][i])
                dij = distance_matrix[route_change[-1]][i]
                cij.append([0.3*Tij+0.3*dij+0.4*vij,i,ltc])
            else:
                #print 'over depot due time'
                continue

        else:
            #print 'over capacity'
            continue
    #print 'length of cij:',len(cij)
    if len(cij) != 0:
        cij.sort()
        route_change.append(cij[0][1])
        leavetimechange.append(cij[0][2])
        cus.remove(cij[0][1])
        #print cij[0]
    else:
        route_append = 1
        route_change.append(0)
        route.append(route_change)
        leavetime.append(leavetimechange)
route_change.append(0)
route.append(route_change)
leavetime.append(leavetimechange)



In [10]:
print 'If we can severice over due time'
print 'Number of cars is',len(route)

print 'Routes are :'
for indexi,i in enumerate(route):
    print indexi,i

If we can severice over due time
Number of cars is 23
Routes are :
0 [0, 59, 42, 98, 16, 85, 43, 91, 100, 0]
1 [0, 92, 5, 44, 38, 97, 17, 93, 0]
2 [0, 14, 15, 40, 50, 68, 25, 0]
3 [0, 45, 83, 88, 49, 60, 89, 58, 0]
4 [0, 63, 62, 11, 90, 10, 32, 77, 0]
5 [0, 72, 2, 73, 22, 56, 4, 80, 0]
6 [0, 33, 29, 51, 3, 54, 70, 0]
7 [0, 36, 47, 19, 8, 46, 48, 0]
8 [0, 27, 69, 30, 81, 34, 35, 0]
9 [0, 95, 21, 87, 26, 24, 0]
10 [0, 28, 12, 76, 79, 20, 1, 0]
11 [0, 39, 23, 67, 55, 0]
12 [0, 65, 71, 9, 66, 0]
13 [0, 31, 7, 37, 13, 0]
14 [0, 52, 99, 6, 96, 0]
15 [0, 82, 18, 84, 0]
16 [0, 75, 53, 74, 0]
17 [0, 64, 0]
18 [0, 61, 86, 0]
19 [0, 78, 0]
20 [0, 41, 0]
21 [0, 57, 0]
22 [0, 94, 0]
